<a href="https://colab.research.google.com/github/lkerroum/book_recomendation_knn/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BOOK RECOMMENDATION ENGINE USING KNN

###Instructions
In this challenge, you will create a book recommendation algorithm using K-Nearest Neighbors.

You will use the Book-Crossings dataset. This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users.

After importing and cleaning the data, use NearestNeighbors from sklearn.neighbors to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures the distance to determine the “closeness” of instances.

Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:



In [ ]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

should return:

In [ ]:
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301],
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479],
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]


Notice that the data returned from get_recommends() is a list. The first element in the list is the book title passed into the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed into the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

###Load libraries

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

###Load data

In [7]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-07-22 07:23:50--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   157MB/s    in 0.2s    

2024-07-22 07:23:50 (157 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

###Import into dataframes and clean

In [33]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})


# data cleaning
df_books.dropna(inplace=True)
df_ratings.dropna(inplace=True)

# drop users with less than 200 ratings
user_counts = df_ratings['user'].value_counts()
valid_users = user_counts[user_counts >= 200].index
df_ratings_cl = df_ratings[df_ratings['user'].isin(valid_users)]

# drop books with less than 100 ratings
book_counts = df_ratings['isbn'].value_counts()
valid_books = book_counts[book_counts >= 100].index
df_ratings_cl = df_ratings_cl[df_ratings_cl['isbn'].isin(valid_books)]
df_books = df_books[df_books['isbn'].isin(valid_books)]

###Prepare dataset for KNN and build model

In [80]:
# prepare df_ratings for nearest neighbors algorithm

ratings_matrix = df_ratings_cl.pivot_table(index='isbn', columns='user', values='rating').fillna(0)
ratings_matrix.index = ratings_matrix.join(df_books.set_index('isbn'))['title']
nbrs = NearestNeighbors(algorithm='auto', radius=1.0, metric='cosine').fit(ratings_matrix.values)


###Create get_recommends() function

In [82]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    if book in ratings_matrix.index:
        dist, recommended_books_indices = nbrs.kneighbors([ratings_matrix.loc[book].values], n_neighbors=6)
        recommended_books = pd.DataFrame({
            'title'   : ratings_matrix.iloc[recommended_books_indices[0]].index.values,
            'distance': dist[0]
            }).sort_values(by='distance', ascending=False).head(5).values

        return [book, recommended_books]

    else:
        print(f"{book} not found in the ratings matrix.")


###Check test cases

In [81]:
book = "The Queen of the Damned (Vampire Chronicles (Paperback))"
dist, recommended_books_indices = nbrs.kneighbors([ratings_matrix.loc[book].values], n_neighbors=6)
recommended_books = pd.DataFrame({
    'title'   : ratings_matrix.iloc[recommended_books_indices[0]].index.values,
    'distance': dist[0]
    }).sort_values(by='distance', ascending=False).head(5).values
print(recommended_books)

[['Catch 22' 0.793983519077301]
 ['The Witching Hour (Lives of the Mayfair Witches)' 0.7448656558990479]
 ['Interview with the Vampire' 0.7345068454742432]
 ['The Tale of the Body Thief (Vampire Chronicles (Paperback))'
  0.5376338362693787]
 ['The Vampire Lestat (Vampire Chronicles, Book II)' 0.5178411602973938]]


In [83]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
      print(2)
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
      print(3)
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
